In [1]:
# Importing necessary models
import warnings
warnings.filterwarnings('ignore')

import smtplib
import pandas as pd
import numpy as np
import datetime as dt

import os
from sqlalchemy import *
from sqlalchemy import create_engine

main_dir = os.getcwd()

dbs_dir = 'C:\\Users\\Fang\\Desktop\\Python Trading\\Trading\\Data\\DBs'

os.chdir('C:\\Users\\Fang\\Desktop\\Python Trading\\Trading\\Trading\\Modules\\DataCollection')

from alphavantage import *
from yahoo_query import *

os.chdir('C:\\Users\\Fang\\Desktop\\Python Trading\\Trading\\Trading\\Modules\\Options')
from optionsFunctions import *

vix_options_dir = 'D:\\Options Data\\VIX'

cleaned_dir = 'D:\\Options Data\\Monthly DBs'
#'vix_2018-07-30

In [2]:
def monthly_spx_intraday_prices(curr_date):
    spx_dir = 'D:\\Options Data\\SPX Intraday'
    os.chdir(spx_dir)
    spx_intraday_dfs = []
    curr_spx_price_files = list(filter(lambda x: curr_date.strftime("%Y-%m") in x, os.listdir('.')))
    for filename in curr_spx_price_files:
        if filename.endswith(".csv"):
            curr_df = pd.read_csv(filename, index_col = 0).dropna()
            curr_df.index = pd.to_datetime(curr_df.index)
            curr_df = curr_df[curr_df.index.time >= dt.time(9,30)]
            spx_intraday_dfs.append(curr_df)
        else:
            continue
    return pd.concat(spx_intraday_dfs)

def monthly_spx_options(curr_date):
    spx_options_dir = 'D:\\Options Data\\SPX'
    os.chdir(spx_options_dir)
    spx_calls_dfs = []
    spx_puts_dfs = []
    curr_spx_options_files = list(filter(lambda x: curr_date.strftime("%Y-%m") in x, os.listdir('.')))
    keep_columns = ['Strike','Expiry','Type','Symbol','Last','Bid','Ask','IV','Underlying_Price']
    for filename in curr_spx_options_files:
        if filename.endswith(".csv"):
            curr_options = pd.read_csv(filename, index_col = 0).set_index('Quote_Time')[keep_columns]
            curr_options.index = pd.to_datetime(curr_options.index) - dt.timedelta(hours = 4)
            curr_options.index = curr_options.index.map(lambda x: x.replace(second=0))
            curr_options['Expiry'] = pd.to_datetime(curr_options.Expiry)
            curr_options['DTE'] = (curr_options.Expiry - curr_options.index).dt.days + 1
            curr_options['Mid'] = (curr_options['Ask'] + curr_options['Bid'])/2
            greeks = all_greeks(curr_options, interest_rate = 0.023869, q = 0, year = 252)
            greeks.index = pd.to_datetime(curr_options.index)
            puts = greeks[greeks.Type == 'put']
            calls = greeks[greeks.Type == 'call']
            spx_calls_dfs.append(calls)
            spx_puts_dfs.append(puts)
        else:
            continue
    puts = pd.concat(spx_puts_dfs)
    calls = pd.concat(spx_calls_dfs)
    return puts, calls

In [3]:
date_range = [dt.datetime(2019, 1, 1)]

for curr_date in date_range:
    os.chdir(cleaned_dir)
    latest_month = curr_date.month
    db_name = 'sqlite:///intraday_{0}-{1}.db'.format(curr_date.year,latest_month)
    
    options_engine = create_engine(db_name, echo=False)
    
    spx_intraday = monthly_spx_intraday_prices(curr_date)
    puts, calls = monthly_spx_options(curr_date)
    
    puts.to_sql('spx_puts', con=options_engine, if_exists='append', index_label = 'quoteTime')
    calls.to_sql('spx_calls', con=options_engine, if_exists='append', index_label = 'quoteTime')
    spx_intraday.to_sql('spx_prices', con=options_engine, if_exists='append')
    print(curr_date)

2019-01-01 00:00:00


In [4]:
spx_ticker = '^GSPC'
spx = yahoo_query(spx_ticker, dt.datetime(2018,8,1))
spx.hist_prices_query()
spx = spx.hist_prices 
spx.columns = [col.replace('^', '') for col in spx.columns.tolist()]

vix_ticker = '^VIX'
vix = yahoo_query(vix_ticker, dt.datetime(2018,8,1))
vix.hist_prices_query()
vix = vix.hist_prices 
vix.columns = [col.replace('^', '') for col in vix.columns.tolist()]

df = spx[['GSPC_open','GSPC_close']].join(vix[['VIX_open']])
df['LB'] = df['GSPC_open']*(1 - df['VIX_open']/(100*np.sqrt(252)))
df['UB'] = df['GSPC_open']*(1 + df['VIX_open']/(100*np.sqrt(252)))

In [83]:
spx_options_dir = 'D:\\Options Data\\SPX'
os.chdir(spx_options_dir)
spx_calls_dfs = []
spx_puts_dfs = []
curr_spx_options_files = os.listdir('.')


keep_columns = ['Strike','Expiry','Type','Symbol','Last','Bid','Ask','IV','Underlying_Price']

dates = []

bod_put_credits = []
bod_call_credits = []

eod_put_credits = []
eod_call_credits = []

for curr_date, row in df.iterrows():
    curr_date_string = curr_date.strftime("%Y-%m-%d")
    curr_filename_bod = 'spx_{}-06-35.csv'.format(curr_date_string)
    curr_filename_eod = 'spx_{}-13-00.csv'.format(curr_date_string)
    
    try:
        curr_options_bod = pd.read_csv(curr_filename_bod, index_col = 0).set_index('Quote_Time')[keep_columns]
        curr_options_bod.index = pd.to_datetime(curr_options_bod.index) - dt.timedelta(hours = 4)
        curr_options_bod.index = curr_options_bod.index.map(lambda x: x.replace(second=0))
        curr_options_bod['Expiry'] = pd.to_datetime(curr_options_bod.Expiry)
        curr_options_bod['DTE'] = (curr_options_bod.Expiry - curr_options_bod.index).dt.days + 1
        curr_options_bod['Mid'] = (curr_options_bod['Ask'] + curr_options_bod['Bid'])/2

        curr_options_bod = curr_options_bod[curr_options_bod.DTE == curr_options_bod.sort_values('DTE').iloc[0,9]]

        put_spread_bod = curr_options_bod[(curr_options_bod.Type == 'put') & 
                                      (curr_options_bod.Strike <= row.LB)].sort_values('Strike').tail(2)
        call_spread_bod = curr_options_bod[(curr_options_bod.Type == 'call') & 
                                       (curr_options_bod.Strike >= row.UB)].sort_values('Strike').head(2)

        put_credit_bod = put_spread_bod.Mid.diff().values[-1]
        call_credit_bod = abs(call_spread_bod.Mid.diff()).values[-1]

        curr_options_eod = pd.read_csv(curr_filename_eod, index_col = 0).set_index('Quote_Time')[keep_columns]
        curr_options_eod.index = pd.to_datetime(curr_options_eod.index) - dt.timedelta(hours = 4)
        curr_options_eod.index = curr_options_eod.index.map(lambda x: x.replace(second=0))
        curr_options_eod['Expiry'] = pd.to_datetime(curr_options_eod.Expiry)
        curr_options_eod['DTE'] = (curr_options_eod.Expiry - curr_options_eod.index).dt.days + 1
        curr_options_eod['Mid'] = (curr_options_eod['Ask'] + curr_options_eod['Bid'])/2

        put_credit_eod = curr_options_eod[curr_options_eod.Symbol.isin(put_spread_bod.Symbol.tolist())].iloc[1,10] - curr_options_eod[curr_options_eod.Symbol.isin(put_spread_bod.Symbol.tolist())].iloc[0,10]
        call_credit_eod = curr_options_eod[curr_options_eod.Symbol.isin(call_spread_bod.Symbol.tolist())].iloc[0,10] - curr_options_eod[curr_options_eod.Symbol.isin(call_spread_bod.Symbol.tolist())].iloc[1,10]

        dates.append(curr_date)
        bod_put_credits.append(put_credit_bod)
        bod_call_credits.append(call_credit_bod)
        eod_put_credits.append(put_credit_eod)
        eod_call_credits.append(call_credit_eod)
    except:
        continue

In [84]:
pd.DataFrame({'bod_puts':bod_put_credits,
              'bod_calls':bod_call_credits,
              'eod_puts':eod_put_credits,
              'eod_calls':eod_call_credits}, index = dates).to_csv('spreads.csv')